In [17]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
import pickle

import glob, os    


In [21]:
irc = pd.read_csv('pcap_files/irc1.pcap.csv')
http = pd.read_csv('pcap_files/output.pcap.csv')
tor = pd.read_csv('pcap_files/tor1.pcap.csv')
torrentz = pd.read_csv('pcap_files/torrentz.pcap.csv')
common = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "pcap_files/*.csv"))))

tor['target'] = 6
http['target'] = 1
torrentz['target'] = 2
irc['target'] = 5

common_target = pd.concat([irc,http,tor,torrentz])['target']

In [22]:
le = preprocessing.LabelEncoder()
scaler = preprocessing.MinMaxScaler()

irc=irc.fillna(0)
tor=tor.fillna(0)
torrentz=torrentz.fillna(0)
http=http.fillna(0)

irc = irc.apply(le.fit_transform)
irc = scaler.fit_transform(irc[list(irc.columns)].to_numpy())
# irc = irc.apply(lambda x: scaler.fit_transform(x.values.reshape(1, -1)))

tor = tor.apply(le.fit_transform)
tor = scaler.fit_transform(tor[list(tor.columns)].to_numpy())

torrentz = torrentz.apply(le.fit_transform)
torrentz = scaler.fit_transform(torrentz[list(torrentz.columns)].to_numpy())

http = http.apply(le.fit_transform)
http = scaler.fit_transform(http[list(http.columns)].to_numpy())



In [ ]:
#Final 
le = preprocessing.LabelEncoder()
scaler = preprocessing.MinMaxScaler()
common1 = common
common1 = common1.apply(le.fit_transform)
common1 = scaler.fit_transform(common1[list(common1.columns)].to_numpy())

In [35]:
# HyperParameter Optimization


clf = RandomForestClassifier( random_state=0)
X_train, X_test, y_train, y_test = train_test_split(common1, common_target.to_numpy(), test_size=0.4)
clf.fit(X_train,y_train)

param_grid = [
{
 'n_estimators': [x for x in range(1,25)], 
 'max_depth': [x for x in range(1,50)], 
}
]

grid_search_forest = GridSearchCV(clf, param_grid, cv=4, scoring='neg_mean_squared_error')
grid_search_forest.fit(common1, common_target.to_numpy())
cvres = grid_search_forest.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)




0.6454972243679028 {'max_depth': 1, 'n_estimators': 1}
0.6454972243679028 {'max_depth': 1, 'n_estimators': 2}
0.6454972243679028 {'max_depth': 1, 'n_estimators': 3}
0.6454972243679028 {'max_depth': 1, 'n_estimators': 4}
0.7071067811865476 {'max_depth': 1, 'n_estimators': 5}
0.5400617248673216 {'max_depth': 1, 'n_estimators': 6}
0.5400617248673216 {'max_depth': 1, 'n_estimators': 7}
0.5400617248673216 {'max_depth': 1, 'n_estimators': 8}
0.5400617248673216 {'max_depth': 1, 'n_estimators': 9}
0.5400617248673216 {'max_depth': 1, 'n_estimators': 10}
0.5400617248673216 {'max_depth': 1, 'n_estimators': 11}
0.408248290463863 {'max_depth': 1, 'n_estimators': 12}
0.5400617248673216 {'max_depth': 1, 'n_estimators': 13}
0.408248290463863 {'max_depth': 1, 'n_estimators': 14}
0.408248290463863 {'max_depth': 1, 'n_estimators': 15}
0.408248290463863 {'max_depth': 1, 'n_estimators': 16}
0.408248290463863 {'max_depth': 1, 'n_estimators': 17}
0.28867513459481287 {'max_depth': 1, 'n_estimators': 18}
0.28

In [36]:
#Feature Selection 
clf = SelectFromModel(RandomForestClassifier(n_estimators = 18, max_depth=9, random_state=0), max_features=30)

# clf.fit(irc, len(irc)*[5])
# clf.fit(tor, len(tor)*[5])
# clf.fit(http, len(http)*[5])
# clf.fit(torrentz, len(torrentz)*[5])
clf.fit(common1, common_target.to_numpy())
feature_index = clf.get_support()

print(common.columns[feature_index])

['ndpi_proto_num', 'src2dst_packets', 'src2dst_bytes', 'dst2src_packets', 'dst2src_bytes', 'data_ratio', 'iat_flow_min', 'iat_flow_avg', 'iat_flow_max', 'iat_flow_stddev', 'iat_c_to_s_min', 'iat_c_to_s_avg', 'iat_c_to_s_max', 'iat_c_to_s_stddev', 'iat_s_to_c_min', 'iat_s_to_c_avg', 'iat_s_to_c_max', 'iat_s_to_c_stddev', 'pktlen_c_to_s_min', 'pktlen_c_to_s_avg', 'pktlen_c_to_s_max', 'pktlen_c_to_s_stddev', 'pktlen_s_to_c_min', 'pktlen_s_to_c_avg', 'pktlen_s_to_c_max', 'pktlen_s_to_c_stddev']


In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

# https://vast.ai/console/create/ 